# SchNet S2EF training example

The purpose of this notebook is to demonstrate some of the basics of the Open Catalyst Project's (OCP) codebase and data. In this example, we will train a schnet model for predicting the energy and forces of a given structure (S2EF task). First, ensure you have installed the OCP ocp repo and all the dependencies according to the [README](https://github.com/Open-Catalyst-Project/ocp/blob/master/README.md).

Disclaimer: This notebook is for tutorial purposes, it is unlikely it will be practical to train baseline models on our larger datasets using this format. As a next step, we recommend trying the command line examples. 

## Imports

In [1]:
import sys
import torch
import os
sys.path.append(r"D:\Train_thử\fairchem-tio2-s2ef")

from ocpmodels.trainers import ForcesTrainer
from ocpmodels import models
from ocpmodels.common import logger
from ocpmodels.common.utils import setup_logging
setup_logging()

D:\Train_thử\fairchem-tio2-s2ef\ocpmodels\models\scn\spherical_harmonics.py:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  _Jd = torch.load(os.path.join(os.path.dirname(_

In [2]:
# a simple sanity check that a GPU is available
if torch.cuda.is_available():
    print("True")
else:
    print("False")

False


## The essential steps for training an OCP model

1) Download data

2) Preprocess data (if necessary)

3) Define or load a configuration (config), which includes the following
   
   - task
   - model
   - optimizer
   - dataset
   - trainer

4) Train

5) Depending on the model/task there might be intermediate relaxation step

6) Predict

## Dataset

This examples uses the LMDB generated from the following [tutorial](http://laikapack.cheme.cmu.edu/notebook/open-catalyst-project/mshuaibi/notebooks/projects/ocp/docs/source/tutorials/lmdb_dataset_creation.ipynb). Please run that notebook before moving on. Alternatively, if you have other LMDBs available you may specify that instead.

In [3]:
# set the path to your local lmdb directory
# train_src = "s2ef"

In [3]:
from ocpmodels.datasets import TrajectoryLmdbDataset
import numpy as np
import copy

train_src = r"D:\Train_thử\fairchem-tio2-s2ef\tio2_s2ef\data\tio2_lmdb\200k"

train_dataset = TrajectoryLmdbDataset({"src": train_src})

energies = []
for data in train_dataset:
  energies.append(data.y)

mean = np.mean(energies)
stdev = np.std(energies)

c:\Users\admin\anaconda3\envs\tio2-s2ef\lib\site-packages\IPython\core\interactiveshell.py:3550: UserWarning: TrajectoryLmdbDataset is deprecated and will be removed in the future.Please use 'LmdbDataset' instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


## Define config

For this example, we will explicitly define the config; however, a set of default config files exists in the config folder of this repository. Default config yaml files can easily be loaded with the `build_config` util (found in `ocp/ocpmodels/common/utils.py`). Loading a yaml config is preferrable when launching jobs from the command line. We have included our best models' config files [here](https://github.com/Open-Catalyst-Project/ocp/tree/master/configs/s2ef).

**Task** 

In [4]:
task = {
    'dataset': 'trajectory_lmdb', # dataset used for the S2EF task
    'description': 'Regressing to energies and forces for DFT trajectories from OCP',
    'type': 'regression',
    'metric': 'mae',
    'labels': ['potential energy'],
    'grad_input': 'atomic forces',
    'train_on_free_atoms': True,
    'eval_on_free_atoms': True
}

**Model** - SchNet for this example

In [5]:
model = {
    'name': 'gemnet_t',
    "num_spherical": 7,
    "num_radial": 128,
    "num_blocks": 3,
    "emb_size_atom": 512,
    "emb_size_edge": 512,
    "emb_size_trip": 64,
    "emb_size_rbf": 16,
    "emb_size_cbf": 16,
    "emb_size_bil_trip": 64,
    "num_before_skip": 1,
    "num_after_skip": 2,
    "num_concat": 1,
    "num_atom": 3,
    "cutoff": 6.0,
    "max_neighbors": 50,
    "rbf": {"name": "gaussian"},
    "envelope": {
      "name": "polynomial",
      "exponent": 5,
    },
    "cbf": {"name": "spherical_harmonics"},
    "extensive": True,
    "otf_graph": True,
    "output_init": "HeOrthogonal",
    "activation": "silu",
    "scale_file": "D:\\Train_thử\\fairchem-tio2-s2ef\\configs\\s2ef\\all\\gemnet\\scaling_factors\\gemnet-dT.json",
    "direct_forces": True,
}

**Optimizer**

In [6]:
optimizer = {
    'batch_size': 8,         # originally 32
    'eval_batch_size': 1,    # originally 32
    'num_workers': 0,
    'lr_initial': 5.e-4,
    'optimizer': 'AdamW',
    'optimizer_params': {"amsgrad": True},
    'scheduler': "ReduceLROnPlateau",
    'mode': "min",
    'factor': 0.8,
    'patience': 3,
    'max_epochs': 1,         # used for demonstration purposes
    'force_coefficient': 100,
    'ema_decay': 0.999,
    'clip_grad_norm': 10,
    'loss_energy': 'mae',
    'loss_force': 'l2mae',
}

**Dataset**

For simplicity, `train_src` is used for all the train/val/test sets. Feel free to update with the actual S2EF val and test sets, but it does require additional downloads and preprocessing. If you desire to normalize your targets, `normalize_labels` must be set to `True` and corresponding `mean` and `stds` need to be specified. These values have been precomputed for you and can be found in any of the [`base.yml`](https://github.com/Open-Catalyst-Project/ocp/blob/master/configs/s2ef/20M/base.yml#L5-L9) config files.

In [7]:
# Dataset
dataset = [
  {'src': train_src,
   'normalize_labels': True,
   "target_mean": mean,
   "target_std": stdev,
   "grad_target_mean": 0.0,
   "grad_target_std": stdev
   }, # train set 
  {'src': train_src},
]

# dataset = [
# {'src': train_src, 'normalize_labels': False}, # train set 
# {'src': train_src}, # val set (optional)
# {'src': train_src} # test set (optional - writes predictions to disk)
# ]

**Trainer**

Use the `ForcesTrainer` for the S2EF and IS2RS tasks, and the `EnergyTrainer` for the IS2RE task 

In [8]:
trainer = ForcesTrainer(
    task=task,
    model=copy.deepcopy(model), # copied for later use, not necessary in practice.
    dataset=dataset,
    optimizer=optimizer,
    identifier="S2EF-example",
    run_dir="./", # directory to save results if is_debug=False. Prediction files are saved here so be careful not to override!
    is_debug=False, # if True, do not save checkpoint, logs, or results
    # is_vis=False,
    print_every=5,
    seed=0, # random seed to use
    logger="tensorboard", # logger of choice (tensorboard and wandb supported)
    local_rank=0,
    amp=True, # use PyTorch Automatic Mixed Precision (faster training and less memory usage),
)

amp: true
cmd:
  checkpoint_dir: ./checkpoints\2026-01-31-16-16-48-S2EF-example
  commit: 5571d3b
  identifier: S2EF-example
  logs_dir: ./logs\tensorboard\2026-01-31-16-16-48-S2EF-example
  print_every: 5
  results_dir: ./results\2026-01-31-16-16-48-S2EF-example
  seed: 0
  timestamp_id: 2026-01-31-16-16-48-S2EF-example
dataset:
  grad_target_mean: 0.0
  grad_target_std: !!python/object/apply:numpy._core.multiarray.scalar
  - &id001 !!python/object/apply:numpy.dtype
    args:
    - f8
    - false
    - true
    state: !!python/tuple
    - 3
    - <
    - null
    - null
    - null
    - -1
    - -1
    - 0
  - !!binary |
    3TJ86Am2bEA=
  normalize_labels: true
  src: "D:\\Train_th\u1EED\\fairchem-tio2-s2ef\\tio2_s2ef\\data\\tio2_lmdb\\200k"
  target_mean: !!python/object/apply:numpy._core.multiarray.scalar
  - *id001
  - !!binary |
    GubjwdEPhcA=
  target_std: !!python/object/apply:numpy._core.multiarray.scalar
  - *id001
  - !!binary |
    3TJ86Am2bEA=
gpus: 0
logger: tensorboard

D:\Train_thử\fairchem-tio2-s2ef\ocpmodels\trainers\base_trainer.py:154: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler() if amp else None
c:\Users\admin\anaconda3\envs\tio2-s2ef\lib\site-packages\torch\amp\grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


2026-01-31 16:17:34 (INFO): Loaded GemNetT with 31671825 parameters.


2026-01-31 16:17:34 (WARNING): Model gradient logging to tensorboard not yet supported.


## Check the model

In [10]:
print(trainer.model)

OCPDataParallel(
  (module): GemNetT(
    (radial_basis): RadialBasis(
      (envelope): PolynomialEnvelope()
      (rbf): GaussianSmearing()
    )
    (cbf_basis3): CircularBasisLayer(
      (radial_basis): RadialBasis(
        (envelope): PolynomialEnvelope()
        (rbf): GaussianSmearing()
      )
    )
    (mlp_rbf3): Dense(
      (linear): Linear(in_features=128, out_features=16, bias=False)
      (_activation): Identity()
    )
    (mlp_cbf3): EfficientInteractionDownProjection()
    (mlp_rbf_h): Dense(
      (linear): Linear(in_features=128, out_features=16, bias=False)
      (_activation): Identity()
    )
    (mlp_rbf_out): Dense(
      (linear): Linear(in_features=128, out_features=16, bias=False)
      (_activation): Identity()
    )
    (atom_emb): AtomEmbedding(
      (embeddings): Embedding(83, 512)
    )
    (edge_emb): EdgeEmbedding(
      (dense): Dense(
        (linear): Linear(in_features=1152, out_features=512, bias=False)
        (_activation): ScaledSiLU(
      

## Train

In [9]:
trainer.train()

D:\Train_thử\fairchem-tio2-s2ef\ocpmodels\trainers\forces_trainer.py:330: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=self.scaler is not None):
c:\Users\admin\anaconda3\envs\tio2-s2ef\lib\site-packages\torch\amp\autocast_mode.py:265: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


2026-01-31 16:19:12 (INFO): forcesx_mae: 1.13e+03, forcesy_mae: 2.51e+03, forcesz_mae: 5.32e+03, forces_mae: 2.99e+03, forces_cos: -3.22e-02, forces_magnitude: 6.61e+03, energy_mae: 9.60e+05, energy_force_within_threshold: 0.00e+00, loss: 6.69e+03, lr: 5.00e-04, epoch: 2.27e-01, step: 5.00e+00
2026-01-31 16:20:31 (INFO): forcesx_mae: 3.09e+02, forcesy_mae: 4.68e+02, forcesz_mae: 8.31e+02, forces_mae: 5.36e+02, forces_cos: 4.07e-02, forces_magnitude: 1.16e+03, energy_mae: 2.52e+04, energy_force_within_threshold: 0.00e+00, loss: 5.95e+02, lr: 5.00e-04, epoch: 4.55e-01, step: 1.00e+01
2026-01-31 16:22:09 (INFO): forcesx_mae: 1.49e+02, forcesy_mae: 2.18e+02, forcesz_mae: 3.18e+02, forces_mae: 2.28e+02, forces_cos: 2.37e-02, forces_magnitude: 4.86e+02, energy_mae: 8.93e+03, energy_force_within_threshold: 0.00e+00, loss: 2.52e+02, lr: 5.00e-04, epoch: 6.82e-01, step: 1.50e+01
2026-01-31 16:23:34 (INFO): forcesx_mae: 1.38e+02, forcesy_mae: 1.91e+02, forcesz_mae: 2.37e+02, forces_mae: 1.89e+02

device 0:   0%|          | 0/169 [00:00<?, ?it/s]D:\Train_thử\fairchem-tio2-s2ef\ocpmodels\trainers\base_trainer.py:661: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=self.scaler is not None):
device 0: 100%|██████████| 169/169 [01:32<00:00,  1.83it/s]

2026-01-31 16:25:26 (INFO): forcesx_mae: 301.9076, forcesy_mae: 434.7267, forcesz_mae: 620.1185, forces_mae: 452.2509, forces_cos: -0.0074, forces_magnitude: 951.5119, energy_mae: 34734.3007, energy_force_within_threshold: 0.0000, loss: 580.5457, epoch: 1.0000


### Load Checkpoint
Once training has completed a `Trainer` class, by default, is loaded with the best checkpoint as determined by training or validation (if available) metrics. To load a `Trainer` class directly with a pretrained model, specify the `checkpoint_path` as defined by your previously trained model (`checkpoint_dir`):

In [10]:
checkpoint_path = os.path.join(trainer.config["cmd"]["checkpoint_dir"], "checkpoint.pt")
checkpoint_path

'./checkpoints\\2026-01-31-16-16-48-S2EF-example\\checkpoint.pt'

In [12]:
# model = {
#     'name': 'schnet',
#     'hidden_channels': 1024, # if training is too slow for example purposes reduce the number of hidden channels
#     'num_filters': 256,
#     'num_interactions': 3,
#     'num_gaussians': 200,
#     'cutoff': 6.0
# }

pretrained_trainer = ForcesTrainer(
    task=task,
    model=model,
    dataset=dataset,
    optimizer=optimizer,
    identifier="SchNet-example",
    run_dir="./", # directory to save results if is_debug=False. Prediction files are saved here so be careful not to override!
    is_debug=False, # if True, do not save checkpoint, logs, or results
    # is_vis=False,
    print_every=10,
    seed=0, # random seed to use
    logger="tensorboard", # logger of choice (tensorboard and wandb supported)
    local_rank=0,
    amp=False, # use PyTorch Automatic Mixed Precision (faster training and less memory usage)
)

pretrained_trainer.load_checkpoint(checkpoint_path=checkpoint_path)

amp: false
cmd:
  checkpoint_dir: ./checkpoints\2026-01-31-16-29-36-SchNet-example
  commit: 5571d3b
  identifier: SchNet-example
  logs_dir: ./logs\tensorboard\2026-01-31-16-29-36-SchNet-example
  print_every: 10
  results_dir: ./results\2026-01-31-16-29-36-SchNet-example
  seed: 0
  timestamp_id: 2026-01-31-16-29-36-SchNet-example
dataset:
  grad_target_mean: 0.0
  grad_target_std: !!python/object/apply:numpy._core.multiarray.scalar
  - &id001 !!python/object/apply:numpy.dtype
    args:
    - f8
    - false
    - true
    state: !!python/tuple
    - 3
    - <
    - null
    - null
    - null
    - -1
    - -1
    - 0
  - !!binary |
    3TJ86Am2bEA=
  normalize_labels: true
  src: "D:\\Train_th\u1EED\\fairchem-tio2-s2ef\\tio2_s2ef\\data\\tio2_lmdb\\200k"
  target_mean: !!python/object/apply:numpy._core.multiarray.scalar
  - *id001
  - !!binary |
    GubjwdEPhcA=
  target_std: !!python/object/apply:numpy._core.multiarray.scalar
  - *id001
  - !!binary |
    3TJ86Am2bEA=
gpus: 0
logger:

2026-01-31 16:29:43 (WARNING): Model gradient logging to tensorboard not yet supported.


2026-01-31 16:29:43 (INFO): Loading checkpoint from: ./checkpoints\2026-01-31-16-16-48-S2EF-example\checkpoint.pt


D:\Train_thử\fairchem-tio2-s2ef\ocpmodels\trainers\base_trainer.py:407: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=m

## Predict

If a test has been provided in your config, predictions are generated and written to disk automatically upon training completion. Otherwise, to make predictions on unseen data a `torch.utils.data` DataLoader object must be constructed. Here we reference our test set to make predictions on. Predictions are saved in `{results_file}.npz` in your `results_dir`.

In [13]:
# make predictions on the existing test_loader
predictions = pretrained_trainer.predict(pretrained_trainer.test_loader, results_file="s2ef_results", disable_tqdm=False)

2026-01-31 16:37:29 (INFO): Predicting on test.


AssertionError: 

In [ ]:
energies = predictions["energy"]
forces = predictions["forces"]